In [1]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
plt.gray()

<Figure size 432x288 with 0 Axes>

In [2]:
import sys
sys.path.append('..')

In [3]:
full_archive = np.load('../data_folder/ct/full_archive.npz', allow_pickle = True)

In [4]:
input_images = full_archive["image"]

In [10]:
import prox_tv
from skimage.transform import radon, iradon, iradon_sart
from PIL import Image, ImageDraw

def get_mask(size):
    mask = Image.new('L', size, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((-1, -1) + size, fill=1)
    mask = np.array(mask)
    return mask

def make_window(img, a, b):
    assert(a < b)
    img = np.minimum(255, np.maximum(0, (img - a) / (b - a) * 255))
    return img

def make_fbp(img, size):
    img = np.array(img)
    theta = np.linspace(0., 360., size, endpoint=False)
    sino = radon(img, theta=theta, circle=True)
    return iradon(sino, theta=theta, circle=True)

img = make_window(input_images[0], -1024, 1024)
fbp = make_fbp(img, 64)
tv1 = prox_tv.tv1_2d(fbp, 0.1)

In [ ]:
import skimage.measure

def normalize(src):
    mi = np.min(src)
    ma = np.max(src)
    return (src - mi) / (ma - mi)

def compare(src, result, sum_):
    # src = normalize(src)
    # result = normalize(result)
    psnr = skimage.measure.compare_psnr(src, result)
    ssim = skimage.measure.compare_ssim(src, result)
    sum_[0] += psnr
    sum_[1] += ssim

sum_fbp = {64:[0., 0.], 128:[0., 0.], 256:[0., 0.], 512:[0., 0.]}
sum_sart = {64:[0, 0], 128:[0, 0], 256:[0, 0], 512:[0, 0]}
sum_tv1 = {64:[0, 0], 128:[0, 0], 256:[0, 0], 512:[0, 0]}

for i, img in enumerate(input_images):
    print("step {}".format(i))
    img = make_window(img, -1024, 1024)
    img = normalize(img * get_mask(img.shape))
    #plt.imshow(img)
    #plt.show()
    for size in [64, 128, 256, 512]:
        print("size {}".format(size), end="\r")
        theta = np.linspace(0., 360., size, endpoint=False)
        sino = radon(img, theta=theta, circle=True)
        fbp = iradon(sino, theta=theta, circle=True)
        #plt.imshow(fbp)
        #plt.show()
        sart = iradon_sart(sino, theta=theta)
        tv1 = prox_tv.tv1_2d(fbp, 0.1)
        #plt.imshow(tv1)
        #plt.show()
        compare(img, fbp, sum_fbp[size])
        compare(img, sart, sum_sart[size])
        compare(img, tv1, sum_tv1[size])
    for target in [[sum_fbp, "FBP"], [sum_sart, "SART"], [sum_tv1, "TV1"]]:
        for j in sorted(target[0].keys()):
            print("{}-{} PSNR:{} SSIM:{}".format(target[1], j, target[0][j][0] / (i+1), target[0][j][1] / (i+1)))

step 0
FBP-64 PSNR:20.128395869 SSIM:0.476827566781
FBP-128 PSNR:24.762792783 SSIM:0.661051917788
FBP-256 PSNR:29.2072904364 SSIM:0.850846806372
FBP-512 PSNR:31.9316705508 SSIM:0.935258979129
SART-64 PSNR:24.3322021121 SSIM:0.788097513325
SART-128 PSNR:26.1244653158 SSIM:0.848550408042
SART-256 PSNR:27.5413939643 SSIM:0.89509196414
SART-512 PSNR:28.4859610775 SSIM:0.921782636768
TV1-64 PSNR:24.954450579 SSIM:0.694034080945
TV1-128 PSNR:28.223695164 SSIM:0.856850820188
TV1-256 PSNR:28.5347079368 SSIM:0.867740083569
TV1-512 PSNR:28.5940769615 SSIM:0.868931446361
step 1
FBP-64 PSNR:21.428297634 SSIM:0.53347788072
FBP-128 PSNR:26.4649257858 SSIM:0.724104185116
FBP-256 PSNR:31.471419511 SSIM:0.897979694459
FBP-512 PSNR:33.8382577325 SSIM:0.956947705496
SART-64 PSNR:25.1373299011 SSIM:0.824133472063
SART-128 PSNR:27.2464459336 SSIM:0.885656973691
SART-256 PSNR:28.8490102634 SSIM:0.928414322417
SART-512 PSNR:29.7165716666 SSIM:0.948166851401
TV1-64 PSNR:25.9999835317 SSIM:0.740202097627
TV1-1